# Test Database connection string 

In [104]:
from pymongo import MongoClient

# Replace the uri string with your MongoDB deployment's connection string.

USERNAME = 'tradingbot_admin'
PASSWORD = 'tradingbot_pass'
CLUSTERNAME = 'tb-cluster-0'

connection_uri = "mongodb+srv://{username}:{password}@{cluster_name}.ztadynx.mongodb.net/?retryWrites=true&w=majority"

uri = connection_uri.format(
    username = USERNAME,
    password = PASSWORD,
    cluster_name = CLUSTERNAME 
)

# client = MongoClient(connection_url)
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


# Retrieve data

In [4]:
# set database and collection code goes here
db = client.sample_guides
coll = db.planets

# find code goes here
cursor = coll.find({"hasRings": False})

# iterate code goes here
for doc in cursor:
    print(doc)


{'_id': ObjectId('621ff30d2a3e781873fcb65c'), 'name': 'Mercury', 'orderFromSun': 1, 'hasRings': False, 'mainAtmosphere': [], 'surfaceTemperatureC': {'min': -173, 'max': 427, 'mean': 67}}
{'_id': ObjectId('621ff30d2a3e781873fcb65e'), 'name': 'Mars', 'orderFromSun': 4, 'hasRings': False, 'mainAtmosphere': ['CO2', 'Ar', 'N'], 'surfaceTemperatureC': {'min': -143, 'max': 35, 'mean': -63}}
{'_id': ObjectId('621ff30d2a3e781873fcb661'), 'name': 'Earth', 'orderFromSun': 3, 'hasRings': False, 'mainAtmosphere': ['N', 'O2', 'Ar'], 'surfaceTemperatureC': {'min': -89.2, 'max': 56.7, 'mean': 14}}
{'_id': ObjectId('621ff30d2a3e781873fcb662'), 'name': 'Venus', 'orderFromSun': 2, 'hasRings': False, 'mainAtmosphere': ['CO2', 'N'], 'surfaceTemperatureC': {'min': None, 'max': None, 'mean': 464}}


# Query database
Try one query at a time

In [5]:
# Set database here:
db = client.sample_guides
coll = db.planets

######-------------- Queries ----------------------------------------

## Query using a single criteria
# cursor = coll.find({"hasRings": False})

## Query with multiple criteria (implicit AND)
# cursor = coll.find({"hasRings": False, "mainAtmosphere": "Ar"})

## Query using the 'less than' -- operator $lt (<)
# cursor = coll.find({"surfaceTemperatureC.mean": {"$lt":15}})

## Compound query (AND)
# cursor = coll.find({"surfaceTemperatureC.mean": {"$lt": 15}, "surfaceTemperatureC.min": {"$gt": -100}})

## Compound query on the same field ---> explicit AND
# cursor = coll.find({"$and": [{"orderFromSun": {"$gt": 2}}, {"orderFromSun": {"$lt": 5}}]})

## not the same as when ommiting the $and operator
# cursor = coll.find({"orderFromSun": {"$gt": 2}}, {"orderFromSun": {"$lt": 5}})

## Queries with mutually exclusive criteria -- OR operator
cursor = coll.find(
    {
        "$or": [
            {"orderFromSun": {"$gt": 7}},
            {"orderFromSun": {"$lt": 2}},
        ]
    }
)


######---------------------------------------------------------------

# iterate code goes here
for doc in cursor:
    print(doc)


# # Close the connection to MongoDB when you're done.
# client.close()

{'_id': ObjectId('621ff30d2a3e781873fcb65c'), 'name': 'Mercury', 'orderFromSun': 1, 'hasRings': False, 'mainAtmosphere': [], 'surfaceTemperatureC': {'min': -173, 'max': 427, 'mean': 67}}
{'_id': ObjectId('621ff30d2a3e781873fcb65f'), 'name': 'Neptune', 'orderFromSun': 8, 'hasRings': True, 'mainAtmosphere': ['H2', 'He', 'CH4'], 'surfaceTemperatureC': {'min': None, 'max': None, 'mean': -201}}


# insert - update - delete data

In [10]:
# select database & create collection if non existent:
db = client.sample_guides
coll = db.comets

coll.drop()

######--------------------------------- insert data ----------------------------------------------

docs = [
    {"name": "Halley's Comet", "officialName": "1P/Halley", "orbitalPeriod": 75, "radius": 3.4175, "mass": 2.2e14},
    {"name": "Wild2", "officialName": "81P/Wild", "orbitalPeriod": 6.41, "radius": 1.5534, "mass": 2.3e13},
    {"name": "Comet Hyakutake", "officialName": "C/1996 B2", "orbitalPeriod": 17000, "radius": 0.77671, "mass": 8.8e12},
    ]

result = coll.insert_many(docs)

# verification display the results of your operation
print(result.inserted_ids)




[ObjectId('65df0fe5028ab6f21b8addbf'), ObjectId('65df0fe5028ab6f21b8addc0'), ObjectId('65df0fe5028ab6f21b8addc1')]


### Update data

In [11]:
######--------------------------------- update data ----------------------------------------------

# multiply ($mul) radius field by convertion factor: 1 mile = 1.60934 km
doc = {"$mul": {"radius": 1.60934}}
result = coll.update_many({}, doc)

# display the results of your operation
print("Number of documents updated: ", result.modified_count)


Number of documents updated:  3


### Delete data

In [12]:
######--------------------------------- delete data ----------------------------------------------

# using a query filter to delete documents where their orbitalPeriod is greater than 5 and less than 85
doc = {
    "orbitalPeriod": {
        "$gt": 5,
        "$lt": 85
    }
}

result = coll.delete_many(doc)

# amount deleted code goes here
print("Number of documents deleted: ", result.deleted_count)

######--------------------------------------------------------------------------------------------

Number of documents deleted:  2


# User Admin Credentials

### Set user credentials for ligin on the api

In [101]:
# create database & collection if non existent:
db = client.api_login_credentials
coll = db.users

coll.drop()

######--------------------------------- insert data ----------------------------------------------

docs = [
    {"username": "user_1", "password": "u_one"},
    {"username": "user_2", "password": "u_two"},
    {"username": "user_3", "password": "u_three"},
    {"username": "user_4", "password": "u_four"},
    {"username": "user_5", "password": "u_five"},
    ]

result = coll.insert_many(docs)

# verification display the results of your operation
print(result.inserted_ids)

[ObjectId('65df2d26028ab6f21b8af075'), ObjectId('65df2d26028ab6f21b8af076'), ObjectId('65df2d26028ab6f21b8af077'), ObjectId('65df2d26028ab6f21b8af078'), ObjectId('65df2d26028ab6f21b8af079')]


In [102]:
# create database & collection if non existent:
db = client.api_login_credentials
coll = db.admins

coll.drop()

######--------------------------------- insert data ----------------------------------------------

docs = [
    {"username": "admin_1", "password": "a_one"},
    {"username": "admin_2", "password": "a_two"},
    {"username": "admin_3", "password": "a_three"},
    ]

result = coll.insert_many(docs)

# verification display the results of your operation
print(result.inserted_ids)

[ObjectId('65df2d26028ab6f21b8af07a'), ObjectId('65df2d26028ab6f21b8af07b'), ObjectId('65df2d26028ab6f21b8af07c')]


In [105]:
with MongoClient(uri) as client:
        
    db = client.api_login_credentials
    coll = db.users

    cursor = coll.find()

    # query = 'SELECT * FROM Users;'
    # results = connection.execute(text(query))

    results = list(cursor)

    
    
print(results)


[{'_id': ObjectId('65df2d26028ab6f21b8af075'), 'username': 'user_1', 'password': 'u_one'}, {'_id': ObjectId('65df2d26028ab6f21b8af076'), 'username': 'user_2', 'password': 'u_two'}, {'_id': ObjectId('65df2d26028ab6f21b8af077'), 'username': 'user_3', 'password': 'u_three'}, {'_id': ObjectId('65df2d26028ab6f21b8af078'), 'username': 'user_4', 'password': 'u_four'}, {'_id': ObjectId('65df2d26028ab6f21b8af079'), 'username': 'user_5', 'password': 'u_five'}]


# Upload serialized model

In [13]:
ls "../models"

 Volume in drive C is OS
 Volume Serial Number is 96E8-C330

 Directory of C:\Ramiro\Data Science\0_DataScientest\04_MLOps_project\TradingBotApp_mlops\models

23/02/2024  15:38    <DIR>          .
28/02/2024  11:26    <DIR>          ..
23/02/2024  15:38                 0 .gitkeep
23/02/2024  17:55         3.882.372 model_test.joblib
23/02/2024  15:38                54 model_test_params.json
               3 File(s)      3.882.426 bytes
               2 Dir(s)  136.145.018.880 bytes free


In [14]:
from joblib import dump, load
import bson.binary

# Path to the model file
models_path = "../models/"
model_name = "model_test"
model_filepath = models_path + model_name + ".joblib"

# Load the model
model = load(model_filepath)

# Read the serialized model bytes
with open(model_filepath, 'rb') as f:
    model_bytes = bson.binary.Binary(f.read())

# Access the database and collection
db = client['trading_bot']
collection = db['models']

# Data to insert
doc = {"name": model_name, "model": model_bytes}

# Insert the model bytes into the collection
collection.insert_one(doc)


InsertOneResult(ObjectId('65df1016028ab6f21b8addc2'), acknowledged=True)

### Retrieve the model from cluster

In [16]:
from joblib import dump, load
import bson.binary
from io import BytesIO

# Load the model from MongoDB
db = client['trading_bot']
collection = db['models']
doc = collection.find_one({"name": model_name})

if doc:
    model_bytes = doc["model"]

    # Create a BytesIO object to store the binary data
    model_buffer = BytesIO(model_bytes)

    # Load the model from the BytesIO object - to correctly handle utf-encoding
    model = load(model_buffer)

    # Now you can use the loaded model as needed
    print("Model loaded successfully:", model)
else:
    print("Model not found in the database.")


Model loaded successfully: KNeighborsClassifier(n_neighbors=32)


# Upload Time Series - csv file

In [63]:
import pandas as pd

# Path to the data file
file_path = "../data/raw/"
file_name = "BTCUSDT-1d-raw"
datafile = file_path + file_name + ".csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(datafile)

In [64]:
# df = df.head(10)
df.head()

,openT,open,high,low,close,baseVol,closeT,quoteVol,nbTrade,takerBaseVol,takerQuoteVol
0,1502928000000,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06
1,1503014400000,4285.08,4371.52,3938.77,4108.37,1199.888264,1503100799999,5.086958e+06,5233,972.868710,4.129123e+06
2,1503100800000,4108.37,4184.69,3850.00,4139.98,381.309763,1503187199999,1.549484e+06,2153,274.336042,1.118002e+06
3,1503187200000,4120.98,4211.08,4032.62,4086.29,467.083022,1503273599999,1.930364e+06,2321,376.795947,1.557401e+06
4,1503273600000,4069.13,4119.62,3911.79,4016.00,691.743060,1503359999999,2.797232e+06,3972,557.356107,2.255663e+06


In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2386 entries, 0 to 2385
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   openT          2386 non-null   int64  
 1   open           2386 non-null   float64
 2   high           2386 non-null   float64
 3   low            2386 non-null   float64
 4   close          2386 non-null   float64
 5   baseVol        2386 non-null   float64
 6   closeT         2386 non-null   int64  
 7   quoteVol       2386 non-null   float64
 8   nbTrade        2386 non-null   int64  
 9   takerBaseVol   2386 non-null   float64
 10  takerQuoteVol  2386 non-null   float64
dtypes: float64(8), int64(3)
memory usage: 205.2 KB


In [79]:
df.describe()

,openT,open,high,low,close,baseVol,closeT,quoteVol,nbTrade,takerBaseVol,takerQuoteVol
count,2.386000e+03,2386.000000,2386.000000,2386.000000,2386.000000,2386.000000,2.386000e+03,2.386000e+03,2.386000e+03,2386.000000,2.386000e+03
mean,1.605960e+12,21487.412049,22037.782066,20885.663592,21509.592272,72488.478233,1.606046e+12,1.632706e+09,1.441035e+06,36053.381454,8.090410e+08
std,5.952296e+10,16178.293277,16590.698244,15723.215679,16190.854641,83795.679149,5.952302e+10,2.033669e+09,1.972967e+06,41689.043795,1.010490e+09
min,1.502928e+12,3188.010000,3276.500000,2817.000000,3189.020000,228.108068,1.503014e+12,9.778657e+05,2.153000e+03,56.190141,2.413638e+05
25%,1.554444e+12,8025.940000,8234.150000,7774.500000,8034.482500,29823.917373,1.554530e+12,2.860238e+08,3.049765e+05,15077.497985,1.469227e+08
50%,1.605960e+12,16593.485000,16814.310000,16208.500000,16598.975000,46019.775275,1.606046e+12,7.939103e+08,8.005875e+05,22808.222105,3.860141e+08
75%,1.657476e+12,32861.790000,34092.002500,31238.425000,32927.805000,76701.578827,1.657562e+12,2.273158e+09,1.578868e+06,37926.445910,1.125118e+09
max,1.708992e+12,67525.820000,69000.000000,66222.400000,67525.830000,760705.362783,1.709078e+12,1.746531e+10,1.522359e+07,374775.574085,8.783916e+09


In [40]:
# Connect to MongoDB
db = client['trading_bot']
collection = db['asset_data']

coll.drop() # ensure we start with an empty collection

# Convert DataFrame to a list of dictionaries (one per row)
data = df.to_dict(orient='records')

# Insert the data into the MongoDB collection
collection.insert_many(data)

InsertManyResult([ObjectId('65df186b028ab6f21b8ae71f'), ObjectId('65df186b028ab6f21b8ae720'), ObjectId('65df186b028ab6f21b8ae721'), ObjectId('65df186b028ab6f21b8ae722'), ObjectId('65df186b028ab6f21b8ae723'), ObjectId('65df186b028ab6f21b8ae724'), ObjectId('65df186b028ab6f21b8ae725'), ObjectId('65df186b028ab6f21b8ae726'), ObjectId('65df186b028ab6f21b8ae727'), ObjectId('65df186b028ab6f21b8ae728'), ObjectId('65df186b028ab6f21b8ae729'), ObjectId('65df186b028ab6f21b8ae72a'), ObjectId('65df186b028ab6f21b8ae72b'), ObjectId('65df186b028ab6f21b8ae72c'), ObjectId('65df186b028ab6f21b8ae72d'), ObjectId('65df186b028ab6f21b8ae72e'), ObjectId('65df186b028ab6f21b8ae72f'), ObjectId('65df186b028ab6f21b8ae730'), ObjectId('65df186b028ab6f21b8ae731'), ObjectId('65df186b028ab6f21b8ae732'), ObjectId('65df186b028ab6f21b8ae733'), ObjectId('65df186b028ab6f21b8ae734'), ObjectId('65df186b028ab6f21b8ae735'), ObjectId('65df186b028ab6f21b8ae736'), ObjectId('65df186b028ab6f21b8ae737'), ObjectId('65df186b028ab6f21b8ae7

### Retrieve entire dataset

In [66]:
cursor = collection.find()
df_retieved = pd.DataFrame(list(cursor))

df_retieved.head()

,_id,openT,open,high,low,close,baseVol,closeT,quoteVol,nbTrade,takerBaseVol,takerQuoteVol
0,65df186b028ab6f21b8ae71f,1502928000000,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06
1,65df186b028ab6f21b8ae720,1503014400000,4285.08,4371.52,3938.77,4108.37,1199.888264,1503100799999,5.086958e+06,5233,972.868710,4.129123e+06
2,65df186b028ab6f21b8ae721,1503100800000,4108.37,4184.69,3850.00,4139.98,381.309763,1503187199999,1.549484e+06,2153,274.336042,1.118002e+06
3,65df186b028ab6f21b8ae722,1503187200000,4120.98,4211.08,4032.62,4086.29,467.083022,1503273599999,1.930364e+06,2321,376.795947,1.557401e+06
4,65df186b028ab6f21b8ae723,1503273600000,4069.13,4119.62,3911.79,4016.00,691.743060,1503359999999,2.797232e+06,3972,557.356107,2.255663e+06


### Query DB based on timestamp range
This query inside mongoDB before pulling the data.

In [90]:
from datetime import datetime

# datetime(year, month, day, hour=0, minute=0, second=0, microsecond=0, tzinfo=None)
start_date = datetime(2017, 8, 17)
end_date = datetime(2017, 8, 20)

def to_timestamp(date):
    return int(date.timestamp() * 1000)

def to_date(timestamp):
    # Divide by 1000 to convert milliseconds to seconds
    return datetime.fromtimestamp(timestamp / 1000)  

query = {"openT": {"$gte": to_timestamp(start_date), "$lte": to_timestamp(end_date)}}
results = collection.find(query)

# Convert MongoDB cursor to pandas DataFrame
query_result_df = pd.DataFrame(list(results))

# Analyze or visualize the queried time series data
query_result_df.head()

,_id,openT,open,high,low,close,baseVol,closeT,quoteVol,nbTrade,takerBaseVol,takerQuoteVol
0,65df186b028ab6f21b8ae71f,1502928000000,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06
1,65df186b028ab6f21b8ae720,1503014400000,4285.08,4371.52,3938.77,4108.37,1199.888264,1503100799999,5.086958e+06,5233,972.868710,4.129123e+06
2,65df186b028ab6f21b8ae721,1503100800000,4108.37,4184.69,3850.00,4139.98,381.309763,1503187199999,1.549484e+06,2153,274.336042,1.118002e+06


### Query based on other criteria

In [94]:
query = {'close': {'$gt': 50000}}

# Sorting results
sort_by = [('openT', 1)]  # Sort by 'openT' ascending
cursor = collection.find(query).sort(sort_by)

# Limiting number of results
# limit = 10
# cursor = collection.find().limit(limit)

# Skipping results
# skip = 5
# cursor = collection.find().skip(skip)

# convert to df and display results
df_query = pd.DataFrame(list(cursor))
print(df_query.shape)
df_query.head()

(157, 12)


,_id,openT,open,high,low,close,baseVol,closeT,quoteVol,nbTrade,takerBaseVol,takerQuoteVol
0,65df186b028ab6f21b8aec1f,1613520000000,49133.45,52618.74,48947.00,52119.71,85743.637818,1613606399999,4.365832e+09,2514374,45055.446241,2.294099e+09
1,65df186b028ab6f21b8aec20,1613606400000,52117.67,52530.00,50901.90,51552.60,60758.046954,1613692799999,3.149973e+09,1891592,28893.358623,1.498232e+09
2,65df186b028ab6f21b8aec21,1613692800000,51552.61,56368.00,50710.20,55906.00,79659.778020,1613779199999,4.251168e+09,2339215,40556.020246,2.164871e+09
3,65df186b028ab6f21b8aec22,1613779200000,55906.00,57700.46,53863.93,55841.19,80948.205314,1613865599999,4.551891e+09,2642846,38636.066274,2.172834e+09
4,65df186b028ab6f21b8aec23,1613865600000,55841.19,58352.80,55477.59,57408.57,58166.708511,1613951999999,3.317998e+09,1950533,29506.089085,1.683107e+09


# Close connection
always

In [95]:
# Close the connection to MongoDB when you're done.
client.close()

In [96]:
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Cannot use MongoClient after close
